In [1]:
%run Jansen_And_Rit.py

FileNotFoundError: SC_in_Schaefer-100.csv not found.

In [ ]:
from scipy import signal
%run BOLD_Model.ipynb

# As J&R model already run with downsampling for eeg, need to adjust downsampling rate
adjusted_downsample = int(downsample_bold / downsample_eeg)

# Initial conditions
initial_conditions = np.array([[0.1, 1, 1, 1]] * num_nodes).T

total_bold_downsampled_sims = int(total_downsampled_sims / adjusted_downsample)

# Array to store results
BOLD_vars = np.zeros((total_downsampled_sims + 1 , 4, num_nodes))
BOLD_vars[0] = initial_conditions
BOLD_temp = np.copy(initial_conditions)

# Run simulation using Euler method, NOTE - total_downsampled_sims is the number of timepoints we have firing rates for 
for i in range(total_downsampled_sims):
    # dt has to match the sampling frew of what you pass into the BOLD model, in this case it is the data already 
    # downsampled by eeg freq
    BOLD_temp += (1/eeg_freq) * np.array(balloon_windkessel_ode(BOLD_temp, i))
    BOLD_vars[i] = np.copy(BOLD_temp)

#Downsample by adjusted rate
BOLD_vars = BOLD_vars[::adjusted_downsample]

# Take final half of results as simulation points
BOLD_sim_length = int(1/2 * total_bold_downsampled_sims)
BOLD_vars_result = BOLD_vars[-BOLD_sim_length:]

# Initialize BOLD array - will only take the final BOLD_sim_length timepoints
BOLD_array = np.zeros((BOLD_sim_length - 1, num_nodes))

# Generate BOLD array
q = BOLD_vars_result[:-1, 3, :]
v = BOLD_vars_result[:-1, 2, :]

BOLD_array = V_0 * (k1 * (1 - q) + k2 * (1 - (q / v)) + k3 * (1 - v))

# Pass BOLD signals through bandpass filter

Fmin, Fmax = 0.01, 0.1
tr = 1 / bold_freq
a0, b0 = signal.bessel(3, [2 * tr * Fmin, 2 * tr * Fmax], btype = 'bandpass')
BOLDfilt = signal.filtfilt(a0, b0, BOLD_array[:,:], axis = 0)

# Calculate the FC from the filtered BOLD signal

FC_matrix = np.corrcoef(BOLDfilt.T)

fig = plt.figure(figsize=(8, 6))
plt.imshow(FC_matrix, cmap='RdBu', vmin=-1, vmax=1)
plt.colorbar()
plt.title('Correlation Matrix')
plt.xlabel('Node Index')
plt.ylabel('Node Index')
plt.show()
plt.close(fig)

In [ ]:
# Calculate goodness of fit for BOLD data ####################################################

# Load the empricical (averaged) FC matrix
observed_fc_matrix = np.load("fc_matrices/averaged_fc_matrix.npy")

# Flatten both and use pearson correlation to calculate goodness of fit
pearson_corr = np.corrcoef(np.ravel(FC_matrix), np.ravel(observed_fc_matrix))[0, 1]

print("Pearson correlation coefficient:", pearson_corr)